In [12]:
import numpy as np

test_user_Size = 6858
test_item_Size = 8981
train_user_Size = 29858 - test_user_Size
train_item_Size = 40981 - test_item_Size

id_users=[]
groudTruths=[[] for i in range(train_user_Size)]



# Load the text file and process data
max_user_id = 0
max_item_id = 0
file_path = "data.txt"
counter=-1
with open(file_path, 'r') as data:
    for line in data:
        user_id, *items = line.strip().split()
        user_id = int(user_id)
        if user_id > train_user_Size:
            id_users.append(user_id)
            counter += 1
        max_user_id = max(max_user_id, user_id)
        for item in items:
            item_id = int(item)
            if user_id > train_user_Size and item_id > train_item_Size:
              # set ground truth indexes
                groudTruths[counter].append(item_id)
            max_item_id = max(max_item_id, item_id)

    # create the R matrix
    Matrix_R = np.zeros((max_user_id + 1, max_item_id + 1))

    # reset file position
    data.seek(0)

    # fill R Matrix from data file
    for line in data:
        user_id, *items = line.strip().split()
        user_id = int(user_id)
        for item in items:
            item_id = int(item)
            Matrix_R[user_id, item_id] += 1
print(Matrix_R)
print(Matrix_R.shape)
print(id_users)
print(groudTruths)

[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(29858, 40981)
[23001, 23002, 23003, 23004, 23005, 23006, 23007, 23008, 23009, 23010, 23011, 23012, 23013, 23014, 23015, 23016, 23017, 23018, 23019, 23020, 23021, 23022, 23023, 23024, 23025, 23026, 23027, 23028, 23029, 23030, 23031, 23032, 23033, 23034, 23035, 23036, 23037, 23038, 23039, 23040, 23041, 23042, 23043, 23044, 23045, 23046, 23047, 23048, 23049, 23050, 23051, 23052, 23053, 23054, 23055, 23056, 23057, 23058, 23059, 23060, 23061, 23062, 23063, 23064, 23065, 23066, 23067, 23068, 23069, 23070, 23071, 23072, 23073, 23074, 23075, 23076, 23077, 23078, 23079, 23080, 23081, 23082, 23083, 23084, 23085, 23086, 23087, 23088, 23089, 23090, 23091, 23092, 23093, 23094, 23095, 23096, 23097, 23098, 23099, 23100, 23101, 23102, 23103, 23104, 23105, 23106, 23107, 23108, 23109, 23110, 23111, 23112, 23113, 23114, 23115, 23116, 23117, 23118, 23

In [13]:
from tqdm import tqdm
class MatrixFactorization:
    def __init__(self, num_items, num_users, num_factors, learning_rate, regularization_rate, num_iterations):
        """
          Initialize the Matrix Factorization model.

          Args:
              num_items (int): Number of items.
              num_users (int): Number of users.
              num_factors (int): Number of latent factors.
              learning_rate (float): Learning rate for gradient descent.
              regularization_rate (float): Regularization rate for L2 regularization.
              num_iterations (int): Number of iterations for training.
        """
        self.num_items = num_items
        self.num_users = num_users
        self.num_factors = num_factors
        self.learning_rate = learning_rate
        self.regularization_rate = regularization_rate
        self.num_iterations = num_iterations
        # Initialize Q and P matrices with random values
        # Start your code
        self.Q = np.random.randn(num_items, num_factors)
        self.P = np.random.randn(num_users, num_factors)
        # End your code

    def sigmoid(self, x):
        """
          Compute the sigmoid function.

          Args:
              x (float): Input value.

          Returns:
              float: Sigmoid value.
        """
        return 1 / (1 + np.exp(-x))

    def update_parameters(self, R):
        """
          Update the parameters Q and P using Stochastic Gradient Descent.

          Args:
              R (ndarray): Rating matrix.
        """
        # Start your code
        for j in tqdm(range(self.num_iterations)):
          for u in range(Matrix_R.shape[0]):
                interaction_items = np.where(R[u] > 0)[0]
                for i in interaction_items:
                 if u <= train_user_Size or i <= train_item_Size:
                  error_of_predict = self.sigmoid(1 - self.Q[i].dot(self.P[u].T))

                  # Update Q[i]

                  Q_gradient = ((2*self.regularization_rate*self.Q[i])-self.P[u].T* error_of_predict)
                  self.Q[i] -= self.learning_rate * Q_gradient

                  # Update P[i]
                  P_gradient = ((2*self.regularization_rate*self.P[u])-self.Q[i]* error_of_predict)
                  self.P[u] -= self.learning_rate * P_gradient
        # End your code



    def train(self, R):
        """
        Train the Matrix Factorization model.

        Args:
            R (ndarray): Rating matrix.
        """
        self.update_parameters(R)

    def predict_rating(self, i, u):
        """
        Predict the rating for item i and user u.

        Args:
            i (int): Item index.
            u (int): User index.

        Returns:
            float: Predicted rating.
        """
        # Start your code

        return self.Q[i, :].dot(self.P[u, :].T)

        # End your code

    def evaluate(self, users_list, groundTruth_list, topk=10):
        """
        Evaluate trained model for item i and user u

        Args:
            users_list (list): Users indexes list.
            groundTruth_list (list) : list of items in users test set
            topk (int): threshold for top item selection

        Returns:
            float: sum(Intersection between topk predicted items and user profile in test set / user profile size in test set) / len(users_list)
        """
        # Start your code
        total_accuracy = 0
        counter=0
        for idx, user in enumerate(users_list):
          #calcaulte rating matrix for user and itmes are in test set

            item_range = np.arange(train_item_Size + 1, train_item_Size + test_item_Size)
            predictions = self.predict_rating(item_range, user)
          #get top 10 items
            top_10_items = np.argsort(predictions)[-topk:]
            top_10_items += (train_item_Size+1)
            groundTruth = groundTruth_list[idx]
            if(len(groundTruth)>0):
              counter += 1
              intersection = np.intersect1d(top_10_items, groundTruth)
              total_accuracy += len(intersection) / min(len(groundTruth), 10)

        return total_accuracy / counter
        # End your code

# Set these values according to your dataset
R = Matrix_R  # rating matrix
num_items = max_item_id+1
num_users = max_user_id+1
num_factors = 65
learning_rate = 0.1
regularization_rate = 0.1
num_iterations = 25
model = MatrixFactorization(num_items, num_users, num_factors, learning_rate, regularization_rate, num_iterations)
model.train(R)

# Test prediction for item 0 and user 0

item_index = 0
user_index = 0
prediction = model.predict_rating(item_index, user_index)
print(f"Predicted rating for item {item_index} and user {user_index}: {prediction}")
# Evaluate model for users in test set
user_indexes = id_users
groudTruths = groudTruths
result = model.evaluate(user_indexes, groudTruths)
print(f"Accuracy for model: {result}")

100%|██████████| 25/25 [08:15<00:00, 19.81s/it]


Predicted rating for item 0 and user 0: 2.3767590747780165
Accuracy for model: 0.013580579652008233
